In [ ]:
library(data.table)
library(tidyverse)
library(data.table)
library(tidyverse)
library(tableone)
library(knitr)
library(pROC)
library(glmnet)
library(ggplot2)
library(MASS)

In [ ]:
# load data files into env
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

system(paste0("gsutil cp ", my_bucket, "/path/to/", "grs_collated.tsv", " ."), intern=T)
system(paste0("gsutil cp ", my_bucket, "/path/to/", "recoded_variants_collated_w_IL4.tsv", " ."), intern=T)

system(paste0("gsutil cp ", my_bucket, "/path/to/", "asthma_exac_pheno_pc.tsv", " ."), intern=T)
system(paste0("gsutil cp ", my_bucket, "/path/to/", "asthma_exac_pheno_pc_cut_at_switch_freeze20250117.tsv", " ."), intern=T)
system(paste0("gsutil cp ", my_bucket, "/path/to/", "pre_index_3yr_max_eosinophil.tsv", " ."), intern=T)

# system(paste0("gsutil cp helper_functions.R ", my_bucket, "/path/to/"), intern=T)
system(paste0("gsutil cp ", my_bucket, "/path/to/", "helper_functions.R", " ."), intern=T)

In [ ]:
source("helper_functions.R")

In [ ]:
pheno_pc_old <- fread("asthma_exac_pheno_pc.tsv", data.table=F)
pheno_pc <- fread("asthma_exac_pheno_pc_cut_at_switch_freeze20250117.tsv", data.table=F)
eos <- fread("pre_index_3yr_max_eosinophil.tsv", data.table=F)

grs <- fread("grs_collated.tsv", data.table=F)
recoded_variants <- fread("recoded_variants_collated_w_IL4.tsv", data.table=F)

In [ ]:
summary(pheno_pc_old$post_index_exac_1yr)
summary(pheno_pc$post_index_exac_1yr)

In [ ]:
df_all <- merge(pheno_pc, grs, by.x='person_id', by.y='#IID')
df_all <- merge(df_all, recoded_variants, by.x='person_id', by.y='IID')
df_all <- merge(df_all, eos, by='person_id', all.x=TRUE)

nrow(df_all)

In [ ]:
df <- df_all %>% filter(race_ethnicity=="White-Not Hispanic or Latino")
nrow(df)

In [ ]:
colnames(df)

In [ ]:
snp_name_map <- c(
  "chr2:233757337:A:G_A" = "IFNG.14147.50.3_snp",
  "chr6:170177225:G:T_G" = "IFNGR2.9180.6.3_snp",
  "chr6:32615369:C:T_C" = "IGHE.IGK.IGL.4135.84.2_snp",
  "chr19:53817382:A:G_A" = "IL10.2773.50.2_snp",
  "chr15:68544807:C:T_C" = "IL13RA1.2633.52.2_15_68544807_snp",
  "chr4:186239907:T:G_T" = "IL13RA1.2633.52.2_4_186239907_snp",
  "chr5:24337093:A:G_A" = "IL17A.9170.24.3_snp",
  "chr9:12201384:T:C_T" = "IL22.2778.10.2_snp",
  "chr12:33925202:A:C_A" = "IL6.4673.13.2_snp",
  "chr3:165788464:A:C_A" = "IRF6.9999.1.3_snp",
  "chr16:27363079:A:G_A" = "IL4R.3055.54.2_16_27363079_snp",
  "chr16:27344882:A:G_A" = "IL4R.3055.54.2_16_27344882_snp",
  "chr16:27362859:T:C_T" = "IL4R.3055.54.2_16_27362859_snp",
  "chr16:27362659:C:T_C" = "IL4R.3055.54.2_16_27362659_snp"
)

rename_snps <- function(df, snp_name_map) {
  colnames(df) <- sapply(colnames(df), function(col) {
    if (col %in% names(snp_name_map)) {
      snp_name_map[[col]]
    } else {
      col
    }
  })
  return(df)
}

df <- rename_snps(df, snp_name_map)
colnames(df)

### Prepare risk scores and snps

In [ ]:
scores=grep("\\.",names(df),value=T)
# added <- c("IL5RA.4491.4.2","TNFAIP8.12563.2.3","PGD.4187.49.2","IL1RAP.2630.12.2")
not_in_mgb_analysis <- c("IL1B.3037.62.1", "IGF1.2952.75.2")
scores <- scores[!scores %in% not_in_mgb_analysis]

rnList=grep("_snp",scores,value=T,invert=T)
print(length(rnList))
print(rnList)

snpList=grep("_snp",names(df),value=T)
print(length(snpList))
print(snpList)

In [ ]:
rNorm <- function(x){
  qnorm((rank(x,na.last="keep", ties.method= "first")-0.5)/sum(!is.na(x)))
}

df[,rnList] <- lapply(df[,rnList],function(x){
  rNorm(x)
})

# Regress out the PCs
pcs=paste0("PC",1:10)
for(pr in rnList){

  df[,pr] <- as.numeric(resid(lm(as.formula(paste0(pr,"~",paste0(pcs,collapse = "+"))),data=df)))
  
}

In [ ]:
sapply(scores,function(x){
  hist(df[,x],main=x,xlab="")
})

In [ ]:
# define response variable
df <- df %>% 
  mutate(response = case_when(
    pre_index_exac == 0 & post_index_exac_1yr > 0 ~ 0,
    pre_index_exac == 0 & post_index_exac_1yr == 0 ~ 1,
    TRUE ~ ifelse((post_index_exac_1yr / pre_index_exac) <= 0.5, 1, 0)
  ))

In [ ]:
table(df$response)

In [ ]:
table(df$initial_biologic)

In [ ]:
# make new variable for biologic
df <- df %>% mutate(initial_biologic_recoded=
  case_when(
    initial_biologic =="Dupilumab" ~"Dupilumab",
    initial_biologic == "Omalizumab" ~ "Omalizumab",
    initial_biologic %in% c("Benralizumab","Mepolizumab") ~ "IL5",
    TRUE ~ NA_character_
  )
)
table(df$initial_biologic_recoded)

In [ ]:
df <- df[!is.na(df$initial_biologic_recoded), ]
cat("Total patients:", nrow(df))

In [ ]:
demovars=c("age","female","race_ethnicity","BMI","smoking_status","pre_index_exac","post_index_exac_1yr","post_index_exac")
varsToFactor=c("female","race_ethnicity","smoking_status")


T1=CreateTableOne(vars=demovars,factorVars = varsToFactor,strata="initial_biologic_recoded",data=df)

# print(T1, nonnormal = c("pre_index_exac", "post_index_exac_1yr", "post_index_exac"))
T1_out <- print(T1,quote = FALSE, noSpaces = TRUE, printToggle = FALSE, varLabel = TRUE)

kable(T1_out)

In [ ]:
df_rm0 <- df %>% filter(post_index_exac_1yr > 0)
T1_rm0=CreateTableOne(vars=demovars,factorVars = varsToFactor,strata="initial_biologic_recoded",data=df_rm0)

T1_out_rm0 <- print(T1_rm0,quote = FALSE, noSpaces = TRUE, printToggle = FALSE, varLabel = TRUE,nonnormal=c(c("pre_index_exac", "post_index_exac_1yr", "post_index_exac")))
kable(T1_out_rm0)

In [ ]:
write.table(T1_out, "table_one_aou.txt", sep = "\t", row.names = FALSE, quote = FALSE)
write.table(T1_out_rm0, "table_one_rm0_aou.txt", sep = "\t", row.names = FALSE, quote = FALSE)

In [ ]:
outcome <- "post_index_exac_1yr"
# outcome <- "post_index_exac"

adjustment_vars <- c("age", "female","BMI", "pre_index_exac")
pcs # defined above

strata=names(table(df$initial_biologic_recoded))

### Regression

#### Baseline exacerbation

In [ ]:
summary(df$max_eosinophil_3yr)

In [ ]:
summary(df$PC1)

In [ ]:
sumTab_nb_baseline_rm0 <- data.frame(matrix(nrow=0,ncol=7))

for (predictor in scores) {

  if(predictor %in% rnList){
    adjustment_vars2=c("age","female","BMI","max_eosinophil_3yr")
  } else{
    adjustment_vars2=c("age","female","BMI",pcs,"max_eosinophil_3yr") 
  }
  
  model_table <- call_model(df_rm0, drug_group="all", outcome="pre_index_exac", predictor, model_type="nb",
                            adjustment_vars2, log_offset=NULL)
  
  sumTab_nb_baseline_rm0 <- rbind(sumTab_nb_baseline_rm0, model_table)
  
}


sumTab_nb_baseline_rm0 <- sumTab_nb_baseline_rm0 %>% 
  mutate(adj_p=signif(p.adjust(pval,method = "BH"),2)) %>%
  mutate(pval = signif(pval, 2)) %>%
  # filter(adj_p < 0.05) %>% 
  dplyr::select(outcome,predictor,beta_95CI,pval,adj_p) %>%
  arrange(pval)


sumTab_nb_baseline_rm0 %>% filter(adj_p < 0.1)
sumTab_nb_baseline_rm0 %>% filter(pval < 0.1)

In [ ]:
sumTab_nb_baseline <- data.frame(matrix(nrow=0,ncol=7))

for (predictor in scores) {

  if(predictor %in% rnList){
    adjustment_vars2=c("age","female","BMI","max_eosinophil_3yr")
  } else{
    adjustment_vars2=c("age","female","BMI",pcs,"max_eosinophil_3yr") 
  }
  
  model_table <- call_model(df, drug_group="all", outcome="pre_index_exac", predictor, model_type="nb",
                            adjustment_vars2, log_offset=NULL)
  
  sumTab_nb_baseline <- rbind(sumTab_nb_baseline, model_table)
  
}


sumTab_nb_baseline <- sumTab_nb_baseline %>% 
  mutate(adj_p=signif(p.adjust(pval,method = "BH"),2)) %>%
  mutate(pval = signif(pval, 2)) %>%
  # filter(adj_p < 0.05) %>% 
  dplyr::select(outcome,predictor,beta_95CI,pval,adj_p) %>%
  arrange(pval)


sumTab_nb_baseline %>% filter(adj_p < 0.1)
sumTab_nb_baseline %>% filter(pval < 0.1)

#### Exacerbation Poisson

In [ ]:
sumTab_poisson <- data.frame(matrix(nrow=0,ncol=7))

for(st in strata){

  for (predictor in scores) {
  
#     cat("Running model for predictor:", predictor, "\n")  # Debugging line to track loop progress
  
    if(predictor %in% rnList){
      adjustment_vars2=adjustment_vars
    } else{
      adjustment_vars2=paste0(c(adjustment_vars,pcs),collapse = "+")
    }
    
    model_table <- call_model(df %>% filter(initial_biologic_recoded==st),
                                drug_group = st, outcome, predictor, model_type = "poisson", 
                                adjustment_vars2, log_offset="days_followed_1yr")
    
    sumTab_poisson <- rbind(sumTab_poisson, model_table)
    
  }
}

sumTab_poisson <- sumTab_poisson %>% 
  mutate(adj_p=signif(p.adjust(pval,method = "BH"),2)) %>%
  mutate(pval = signif(pval, 2)) %>%
  # filter(adj_p < 0.05) %>% 
  dplyr::select(drug_group,outcome,predictor,beta_95CI,pval,adj_p) %>%
  arrange(drug_group,adj_p)

sumTab_poisson %>% filter(adj_p < 0.1)

#### Exacerbation NB

In [ ]:
sumTab_nb <- data.frame(matrix(nrow=0,ncol=7))

for(st in strata){

  for (predictor in scores) {
  
#     cat("Running model for predictor:", predictor, "\n")
  
    if(predictor %in% rnList){
      adjustment_vars2=adjustment_vars
    } else{
      adjustment_vars2=paste0(c(adjustment_vars,pcs),collapse = "+")
    }
    
    model_table <- call_model(df %>% filter(initial_biologic_recoded==st),
                                drug_group = st, outcome, predictor, model_type = "nb", 
                                adjustment_vars2, log_offset="days_followed_1yr")
    
    sumTab_nb <- rbind(sumTab_nb, model_table)
    
  }
}

sumTab_nb <- sumTab_nb %>% 
  mutate(adj_p=signif(p.adjust(pval,method = "BH"),2)) %>%
  mutate(pval = signif(pval, 2)) %>%
  # filter(adj_p < 0.05) %>% 
  dplyr::select(drug_group,outcome,predictor,beta_95CI,pval,adj_p) %>%
  arrange(drug_group,adj_p)

write.table(sumTab_nb, "exac_sumTab_nb_aou.txt", sep = "\t", row.names = TRUE, quote = FALSE)

In [ ]:
sumTab_nb_filt <- sumTab_nb %>% filter(pval < 0.1)
sumTab_nb_filt

In [ ]:
pred_short_nb=unique(sumTab_nb_filt$predictor)
length(pred_short_nb)
pred_short_nb

#### Exacerbation NB remove zeros

In [ ]:
sumTab_nb_rm0 <- data.frame(matrix(nrow=0,ncol=7))

for(st in strata){

  for (predictor in scores) {
  
#     cat("Running model for predictor:", predictor, "\n")
  
    if(predictor %in% rnList){
      adjustment_vars2=adjustment_vars
    } else{
      adjustment_vars2=paste0(c(adjustment_vars,pcs),collapse = "+")
    }
    
    model_table <- call_model(df_rm0 %>% filter(initial_biologic_recoded==st),
                                drug_group = st, outcome, predictor, model_type = "nb", 
                                adjustment_vars2, log_offset="days_followed_1yr")
    
    sumTab_nb_rm0 <- rbind(sumTab_nb_rm0, model_table)
    
  }
}

sumTab_nb_rm0 <- sumTab_nb_rm0 %>% 
  mutate(adj_p=signif(p.adjust(pval,method = "BH"),2)) %>%
  mutate(pval = signif(pval, 2)) %>%
  # filter(adj_p < 0.05) %>% 
  dplyr::select(drug_group,outcome,predictor,beta_95CI,pval,adj_p) %>%
  arrange(drug_group,adj_p)

write.table(sumTab_nb_rm0, "exac_sumTab_nb_rm0_aou.txt", sep = "\t", row.names = TRUE, quote = FALSE)

In [ ]:
sumTab_nb_rm0_filt <- sumTab_nb_rm0 %>% filter(pval < 0.1)
sumTab_nb_rm0_filt

In [ ]:
pred_short_nb_rm0=unique(sumTab_nb_rm0_filt$predictor)
length(pred_short_nb_rm0)
pred_short_nb_rm0

#### Responsiveness

In [ ]:
sumTab_log=data.frame(
  drug_group=character(),
  outcome=character(),
  predictor=character(),
  OR=character(),
  p=numeric(),
  AUC=numeric()
)

# Loop through each predictor, fit the model, and calculate AUROC
for(st in strata){
    
  for (predictor in scores) {

      if(predictor %in% rnList){
      adjustment_vars2=adjustment_vars
    } else{
      adjustment_vars2=paste0(c(adjustment_vars,pcs),collapse = "+")
    }
    
    # Fit the model with adjustment variables
  formula <- as.formula(paste("response ~", predictor, "+", paste(adjustment_vars2, collapse = "+")))
  model <- glm(formula, data = df %>% filter(initial_biologic_recoded==st), family = binomial, maxit=500)
  
  if(is.na(coef(model)[predictor])) { next }
 
  # Calculate AUROC
  roc_obj <- roc(model$y, model$fitted.values)
  
   res=data.frame(
    drug_group=st,
    outcome="response",
    predictor=predictor,
    OR=OrCi(model,predictor),
    pval=pvalglm(model,predictor),
    AUC=signif(roc_obj$auc,3)
  )
  
  sumTab_log=rbind(sumTab_log,res)

  }
}


sumTab_log <- sumTab_log %>% 
  mutate(pval = signif(pval, 2)) %>%
  dplyr::select(drug_group,outcome,predictor,OR,pval,AUC) %>%
  arrange(drug_group,desc(AUC))

write.table(sumTab_log, "resp_sumTab_log_aou.txt", sep = "\t", row.names = TRUE, quote = FALSE)

In [ ]:
sumTab_log_filt <- sumTab_log %>% filter(pval < 0.1)
sumTab_log_filt

In [ ]:
df_oma <- df %>% filter(initial_biologic_recoded=="Omalizumab")
model_IL21_oma_resp <- build_model(df_oma, outcome="response", predictor="IL21.7124.18.3", model_type="binomial", adjustment_vars, log_offset=NULL)
roc_IL21_oma_resp <- roc(model_IL21_oma_resp$y, model_IL21_oma_resp$fitted.values)

df_dupi <- df %>% filter(initial_biologic_recoded=="Dupilumab")
model_IL21_dupi_resp <- build_model(df_dupi, outcome="response", predictor="IL21.7124.18.3", model_type="binomial", adjustment_vars, log_offset=NULL)
roc_IL21_dupi_resp <- roc(model_IL21_dupi_resp$y, model_IL21_dupi_resp$fitted.values)

df_IL5 <- df %>% filter(initial_biologic_recoded=="IL5")
model_IL21_IL5_resp <- build_model(df_IL5, outcome="response", predictor="IL21.7124.18.3", model_type="binomial", adjustment_vars, log_offset=NULL)
roc_IL21_IL5_resp <- roc(model_IL21_IL5_resp$y, model_IL21_IL5_resp$fitted.values)


plot(smooth(roc_IL21_oma_resp), col = "#006600", main = "Response predicted by IL21.7124.18.3", cex.main = 1.2, font.main = 1)
plot(smooth(roc_IL21_dupi_resp), col = "#FF6600", add = TRUE)
plot(smooth(roc_IL21_IL5_resp), col = "#0033FF", add = TRUE)

labels <- c(
  paste("IL21 Omalizumab response (AUC =", signif(auc(roc_IL21_oma_resp), 3), ")"),
  paste("IL21 Dupilumab response (AUC =", signif(auc(roc_IL21_dupi_resp), 3), ")"),
  paste("IL21 Mepolizumab response (AUC =", signif(auc(roc_IL21_IL5_resp), 3), ")")
)

legend("bottomright", legend = labels, col = c("#006600", "#FF6600", "#0033FF"), lwd = 2, cex = 0.8, y.intersp = 0.8)

saveRDS(list(roc_IL21_oma_resp = roc_IL21_oma_resp, roc_IL21_dupi_resp = roc_IL21_dupi_resp, roc_IL21_IL5_resp = roc_IL21_IL5_resp), "IL21_roc_list_aou.rds")


In [ ]:
model_IL5RA_IL21_oma_resp <- build_model(df_oma, outcome="response", predictor="IL5RA.13686.2.3 + IL21.7124.18.3", model_type="binomial", adjustment_vars, log_offset=NULL)
roc_IL5RA_IL21_oma_resp <- roc(model_IL5RA_IL21_oma_resp$y, model_IL5RA_IL21_oma_resp$fitted.values)

plot(smooth(roc_IL5RA_IL21_oma_resp), col = "#FF66FF", main = "Response in Omalizumab", cex.main = 1.2, font.main = 1)

labels <- c(
  paste("IL21 + IL5RA Omalizumab response (AUC =", signif(auc(roc_IL5RA_IL21_oma_resp), 3), ")")
)

legend("bottomright", legend = labels, col = c("#FF66FF"), lwd = 2, cex = 0.8, y.intersp = 0.8)

saveRDS(list(roc_IL5RA_IL21_oma_resp = roc_IL5RA_IL21_oma_resp), "IL5RA_IL21_roc_list_aou.rds")

In [ ]:
model_CCL17_IL21_dupi_resp <- build_model(df_dupi, outcome="response", predictor="CCL17.3519.3.2 + IL21.7124.18.3", model_type="binomial", adjustment_vars, log_offset=NULL)
roc_CCL17_IL21_dupi_resp <- roc(model_CCL17_IL21_dupi_resp$y, model_CCL17_IL21_dupi_resp$fitted.values)

plot(smooth(roc_CCL17_IL21_dupi_resp), col = "#006600", main = "Response in Dupilumab", cex.main = 1.2, font.main = 1)

labels <- c(
  paste("IL21 + CCL17 Dupilumab response (AUC =", signif(auc(roc_CCL17_IL21_dupi_resp), 3), ")")
)

legend("bottomright", legend = labels, col = c("#006600"), lwd = 2, cex = 0.8, y.intersp = 0.8)

saveRDS(list(roc_CCL17_IL21_dupi_resp = roc_CCL17_IL21_dupi_resp), "CCL17_IL21_roc_list_aou.rds")

In [ ]:
print(roc.test(roc_IL21_oma_resp, roc_IL5RA_IL21_oma_resp))
print(roc.test(roc_IL21_dupi_resp, roc_CCL17_IL21_dupi_resp))

In [ ]:
model_IL4_oma_resp <- build_model(df_oma, outcome="response", predictor="IL4R.3055.54.2_16_27344882_snp", model_type="binomial", adjustment_vars, log_offset=NULL)
roc_IL4_oma_resp <- roc(model_IL4_oma_resp$y, model_IL4_oma_resp$fitted.values)

model_IL4_dupi_resp <- build_model(df_dupi, outcome="response", predictor="IL4R.3055.54.2_16_27344882_snp", model_type="binomial", adjustment_vars, log_offset=NULL)
roc_IL4_dupi_resp <- roc(model_IL4_dupi_resp$y, model_IL4_dupi_resp$fitted.values)

model_IL4_IL5_resp <- build_model(df_IL5, outcome="response", predictor="IL4R.3055.54.2_16_27344882_snp", model_type="binomial", adjustment_vars, log_offset=NULL)
roc_IL4_IL5_resp <- roc(model_IL4_IL5_resp$y, model_IL4_IL5_resp$fitted.values)

plot(smooth(roc_IL4_oma_resp), col = "#006600", main = "Response predicted by IL4R.3055.54.2_16_27344882 SNP", cex.main = 1.2, font.main = 1)
plot(smooth(roc_IL4_dupi_resp), col = "#FF6600", add = TRUE)
plot(smooth(roc_IL4_IL5_resp), col = "#0033FF", add = TRUE)

labels <- c(
  paste("IL4 SNP Dupilumab response (AUC =", signif(auc(roc_IL4_oma_resp), 3), ")"),
  paste("IL4 SNP Omalizumab response (AUC =", signif(auc(roc_IL4_dupi_resp), 3), ")"),
  paste("IL4 SNP Mepolizumab response (AUC =", signif(auc(roc_IL4_IL5_resp), 3), ")")
)

legend("bottomright", legend = labels, col = c("#006600", "#FF6600", "#0033FF"), lwd = 2, cex = 0.8, y.intersp = 0.8)

saveRDS(list(roc_IL4_oma_resp = roc_IL4_oma_resp, roc_IL4_dupi_resp = roc_IL4_dupi_resp, roc_IL4_IL5_resp = roc_IL4_IL5_resp), "IL4_roc_list_aou.rds")


### Visualize associations

In [ ]:
for(i in 1:nrow(sumTab_nb_rm0_filt)){
  
  grp=sumTab_nb_rm0_filt[[i,"drug_group"]]
  ot=sumTab_nb_rm0_filt[[i,"outcome"]]
  pr=sumTab_nb_rm0_filt[[i,"predictor"]]
  
  # correlation coefficient
  df_subset=df %>% filter(initial_biologic_recoded==grp)
  res_cor=cor.test(df_subset[,pr],df_subset[,ot])
  
  if(pr %in% rnList){
    
    print(
    ggplot(df_subset,
           aes(x=get(pr),y=get(ot)))+
      geom_point()+
      geom_smooth(method="lm")+
      xlab(pr)+ylab(ot)+
      theme_classic()+
      labs(subtitle = paste0("r=",signif(res_cor$estimate,2),", p=",signif(res_cor$p.value,2)))+
      ggtitle(grp)
    
  
    )
    
  } else {
    
     print(
    ggplot(df_subset,
           aes(x=as.factor(get(pr)),y=get(ot)))+
      geom_violin()+
      geom_point()+
      xlab(pr)+ylab(ot)+
      theme_classic()+
      labs(subtitle = paste0("r=",signif(res_cor$estimate,2),", p=",signif(res_cor$p.value,2)))+
      ggtitle(grp)
    
  
    )
    
  }
  
}

In [ ]:

# for responsiveness
for(i in 1:nrow(sumTab_log_filt)){
  
  grp=sumTab_log_filt[[i,"drug_group"]]
  ot=sumTab_log_filt[[i,"outcome"]]
  pr=sumTab_log_filt[[i,"predictor"]]
  
  # correlation coefficient
  df_subset=df %>% filter(initial_biologic_recoded==grp)
  res_ttest=t.test(df_subset[,pr]~df_subset[,ot])
  
  if(pr %in% rnList){
    
    print(
    ggplot(df_subset,
           aes(x=as.factor(get(ot)),y=get(pr)))+
      geom_boxplot()+
      geom_violin()+
      xlab(ot)+ylab(pr)+
      theme_classic()+
      labs(subtitle = paste0("p=",signif(res_ttest$p.value,2)))+
      ggtitle(grp)
    
  
    )
  } else {
    
    
    summary_data <- df %>%
      group_by(get(pr)) %>%
      summarise(
      Mean_Outcome = mean(get(ot)),
      Count = n()
    )

  # Bar plot of mean outcome for each SNP category
  print(
    ggplot(summary_data, aes(x = as.factor(`get(pr)`), y = Mean_Outcome)) +
     geom_bar(stat = "identity", fill = "skyblue", alpha = 0.8) +
      geom_text(aes(label = round(Mean_Outcome, 2)), vjust = -0.5) +
      labs(
        title = grp,
        x = pr,
        y = "Response"
      ) +
    theme_classic()
  )  
  } 
}